## Building and training the model

Import the data from the data processing notebook

In [14]:
%store -r x_train
%store -r x_test
%store -r y_train
%store -r y_test
%store -r yy
%store -r le

Import Deep Learning frameworks

In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics

Using TensorFlow backend.


Build the neural network

In [4]:
num_labels = yy.shape[1]
filter_size = 2

model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [5]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               10496     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                

Use the model to evaluate the test data before the network has been trained. This will show how training the model improves the predictions. (If the model has been trained in this session the accuracy will be high)

In [22]:
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Pre-training accuracy: 91.3793%


Train the model by fitting it to the training data. Go through 100 iterations to train the model

In [9]:
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 1623 samples, validate on 406 samples
Epoch 1/100
1623/1623 [==============================] - 0s 216us/step - loss: 1.9447 - accuracy: 0.8343 - val_loss: 1.3154 - val_accuracy: 0.8990
Epoch 2/100
1623/1623 [==============================] - 0s 215us/step - loss: 1.3706 - accuracy: 0.8392 - val_loss: 0.6475 - val_accuracy: 0.9039
Epoch 3/100
1623/1623 [==============================] - 0s 211us/step - loss: 0.9305 - accuracy: 0.8564 - val_loss: 0.4699 - val_accuracy: 0.9064
Epoch 4/100
1623/1623 [==============================] - 0s 210us/step - loss: 0.6885 - accuracy: 0.8651 - val_loss: 0.3149 - val_accuracy: 0.9163
Epoch 5/100
1623/1623 [==============================] - 0s 213us/step - loss: 0.5344 - accuracy: 0.8792 - val_loss: 0.3551 - val_accuracy: 0.9064
Epoch 6/100
1623/1623 [==============================] - 0s 215us/step - loss: 0.4469 - accuracy: 0.8743 - val_loss: 0.3427 - val_accuracy: 0.9064
Epoch 7/100
1623/1623 [==============================] - 0s 213us/step 

1623/1623 [==============================] - 0s 262us/step - loss: 0.1173 - accuracy: 0.9519 - val_loss: 0.3308 - val_accuracy: 0.9187
Epoch 57/100
1623/1623 [==============================] - 0s 254us/step - loss: 0.1263 - accuracy: 0.9501 - val_loss: 0.3438 - val_accuracy: 0.9089
Epoch 58/100
1623/1623 [==============================] - 0s 251us/step - loss: 0.1089 - accuracy: 0.9538 - val_loss: 0.3337 - val_accuracy: 0.9089
Epoch 59/100
1623/1623 [==============================] - 0s 252us/step - loss: 0.1178 - accuracy: 0.9513 - val_loss: 0.3206 - val_accuracy: 0.9163
Epoch 60/100
1623/1623 [==============================] - 0s 251us/step - loss: 0.1148 - accuracy: 0.9600 - val_loss: 0.3962 - val_accuracy: 0.9064
Epoch 61/100
1623/1623 [==============================] - 0s 248us/step - loss: 0.1240 - accuracy: 0.9532 - val_loss: 0.3651 - val_accuracy: 0.9138
Epoch 62/100
1623/1623 [==============================] - 0s 257us/step - loss: 0.1140 - accuracy: 0.9581 - val_loss: 0.3831 

Evaluate the model on both the train set and the test set. This should give an indication of overfitting.

In [10]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9926062822341919
Testing Accuracy:  0.9137930870056152


To use the classifier, use the dame extract features function as in the previous notebook.

In [11]:
import librosa 
import numpy as np 

def extract_feature(file_name):
   
    try:
        audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None, None

    return np.array([mfccsscaled])

In [12]:
def print_prediction(file_name):
    prediction_feature = extract_feature(file_name) 

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("The predicted class is:", predicted_class[0], '\n') 

    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )

In [17]:
emergency_test_dataset_path = '../Datasets/kaggle/ambulance/sound_1.wav'

print_prediction(emergency_test_dataset_path)

The predicted class is: emergency 

emergency 		 :  1.00000000000000000000000000000000
non_emergency 		 :  0.00000001810440863891926710493863
